# Eurovision's data Preparation
#### This code is used to prepare the scraped data of the Eurovision for analysing

## Importing Libraries

In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale

In [30]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

## Preparing the data

### Eurovision's Events

In [31]:
df_events = pd.read_csv('Eurovision events.csv')

In [32]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         69 non-null     int64  
 1   City         69 non-null     object 
 2   Winner       69 non-null     object 
 3   Participant  69 non-null     object 
 4   Song         24 non-null     object 
 5   Points       68 non-null     float64
 6   Url          69 non-null     object 
dtypes: float64(1), int64(1), object(5)
memory usage: 3.9+ KB


In [33]:
df_events.head()

,Year,City,Winner,Participant,Song,Points,Url
0,2022,Turin,Ukraine,Kalush Orchestra,Stefania,631.0,https://eurovision.tv/event/turin-2022
1,2021,Rotterdam,Italy,Måneskin,Zitti E Buoni,524.0,https://eurovision.tv/event/rotterdam-2021
2,2019,Tel Aviv,Netherlands,Duncan Laurence,Arcade,498.0,https://eurovision.tv/event/tel-aviv-2019
3,2018,Lisbon,Israel,Netta,TOY,529.0,https://eurovision.tv/event/lisbon-2018
4,2017,Kyiv,Portugal,Salvador Sobral,Amar Pelos Dois,758.0,https://eurovision.tv/event/kyiv-2017


In [34]:
df_events.drop(columns = ['Participant','Song','Points','Url'], inplace=True)
df_events = df_events.groupby(by = ['Year','City']).count().reset_index()
df_events.rename(columns = {'City':'Place','Winner':'Nr_Winners'}, inplace=True)
df_events = df_events.set_index('Year')

### Eurovision's Contests

In [35]:
df_contests = pd.read_csv('Eurovision contests.csv')

In [36]:
df_contests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1964 entries, 0 to 1963
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   R/O Sort descending  1964 non-null   int64  
 1   Half                 1964 non-null   object 
 2   Country              1964 non-null   object 
 3   Participant          1964 non-null   object 
 4   Song                 1963 non-null   object 
 5   Points               1913 non-null   float64
 6   Rank                 1964 non-null   object 
 7   Year                 1964 non-null   int64  
 8   Format               1964 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 138.2+ KB


In [37]:
df_contests.head()

,R/O Sort descending,Half,Country,Participant,Song,Points,Rank,Year,Format
0,1,—,Netherlands,Jetty Paerl,De Vogels Van Holland,NaN,2nd,1956,final
1,2,—,Switzerland,Lys Assia,Das Alte Karussell,NaN,2nd,1956,final
2,3,—,Belgium,Fud Leclerc,Messieurs Les Noyés De La Seine,NaN,2nd,1956,final
3,4,—,Germany,Walter Andreas Schwarz,Im Wartesaal Zum Großen Glück,NaN,2nd,1956,final
4,5,—,France,Mathé Altéry,Le Temps Perdu,NaN,2nd,1956,final


In [38]:
df_contests.isna().sum()

R/O Sort descending     0
Half                    0
Country                 0
Participant             0
Song                    1
Points                 51
Rank                    0
Year                    0
Format                  0
dtype: int64

In [39]:
df_contests[df_contests['Song'].isna()]

,R/O Sort descending,Half,Country,Participant,Song,Points,Rank,Year,Format
1059,24,—,Serbia & Montenegro,"Withdrew from the competition, but still voted",NaN,NaN,24th,2006,semi-final


In [40]:
df_contests[df_contests['Points'].isna()].head()

,R/O Sort descending,Half,Country,Participant,Song,Points,Rank,Year,Format
0,1,—,Netherlands,Jetty Paerl,De Vogels Van Holland,NaN,2nd,1956,final
1,2,—,Switzerland,Lys Assia,Das Alte Karussell,NaN,2nd,1956,final
2,3,—,Belgium,Fud Leclerc,Messieurs Les Noyés De La Seine,NaN,2nd,1956,final
3,4,—,Germany,Walter Andreas Schwarz,Im Wartesaal Zum Großen Glück,NaN,2nd,1956,final
4,5,—,France,Mathé Altéry,Le Temps Perdu,NaN,2nd,1956,final


The scores and the scoring method of 1956 is unknown.

In [41]:
df_contests['Format'].value_counts()

final                917
grand-final          456
second-semi-final    248
first-semi-final     244
semi-final            99
Name: Format, dtype: int64

Transforming and preparing the data

In [42]:
df_contests.drop(columns = ['Half'], inplace=True)
df_contests['Rank'] = df_contests['Rank'].replace(to_replace = r'[stndrh]',value = '',regex=True).astype('int64')
df_contests.rename(columns = {'R/O Sort descending':'Order'}, inplace=True)

df_contests['Format'] = df_contests['Format'].replace({'final':'Final','grand-final':'Final','second-semi-final':'Semi-Final',
                                                       'first-semi-final':'Semi-Final','semi-final':'Semi-Final'})

df_contests['Win'] = np.where((df_contests['Format'] == 'Final') & (df_contests['Rank'] == 1), 1, 0)
df_contests['Qualified'] = np.where(df_contests['Participant'].str.contains('qualified'), 1, 0)
df_contests['Participant'] = df_contests['Participant'].str.removesuffix('qualified')

df_contests['Points'] = df_contests['Points'].fillna(0).astype('int64')

df_contests = df_contests.set_index(['Year','Format','Country'])

### Combining Events and Contests

In [43]:
df_full = df_events.join(df_contests).reset_index()

In [44]:
df_full.head()

,Year,Format,Country,Place,Nr_Winners,Order,Participant,Song,Points,Rank,Win,Qualified
0,1956,Final,Netherlands,Lugano,1,1,Jetty Paerl,De Vogels Van Holland,0,2,0,0
1,1956,Final,Switzerland,Lugano,1,2,Lys Assia,Das Alte Karussell,0,2,0,0
2,1956,Final,Belgium,Lugano,1,3,Fud Leclerc,Messieurs Les Noyés De La Seine,0,2,0,0
3,1956,Final,Germany,Lugano,1,4,Walter Andreas Schwarz,Im Wartesaal Zum Großen Glück,0,2,0,0
4,1956,Final,France,Lugano,1,5,Mathé Altéry,Le Temps Perdu,0,2,0,0


In [45]:
df_full.to_csv('Eurovision full.csv', index=False)

### Scores

In [46]:
df_scores = pd.read_csv('Eurovision scores.csv')

In [47]:
df_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27597 entries, 0 to 27596
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        27597 non-null  int64 
 1   Format      27597 non-null  object
 2   Country     27597 non-null  object
 3   From        27597 non-null  object
 4   Score_type  27597 non-null  object
 5   Score       27597 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.3+ MB


In [48]:
df_scores.head()

,Year,Format,Country,From,Score_type,Score
0,1957,final,Belgium,Denmark,Jury,2
1,1957,final,Belgium,Germany,Jury,2
2,1957,final,Belgium,Switzerland,Jury,1
3,1957,final,Luxembourg,Italy,Jury,4
4,1957,final,Luxembourg,Austria,Jury,3


In [49]:
df_scores.isna().sum()

Year          0
Format        0
Country       0
From          0
Score_type    0
Score         0
dtype: int64

In [50]:
df_scores.rename(columns = {'To_country':'Country', 'From_country':'From'}, inplace=True)
df_scores['Format'] = df_scores['Format'].replace({'final':'Final','grand-final':'Final','second-semi-final':'Semi-Final',
                                                   'first-semi-final':'Semi-Final','semi-final':'Semi-Final'})

#### Scores Normalization

During the years, many scoring methods came to use in the Eurovision contests (excluding 1956 in which the scoring method is unknown and therefor can not be examined). Even due a specific scoring method was used at a certain year, the full extend of the scoring range was not always used. To normalize the scores we need to check the min and max scores given in a certain year and use them as the full nue range of scores.

In [51]:
# These variables uses for testing perposes only
# score_1_to_10 = [1957,1958,1959,1960,1961,1967,1968,1969,1970,1974]
# score_1_to_3 = [1962]
# score_1_to_5 = [1963]
# score_1_to_9 = [1964,1965,1966]
# score_2_to_10 = [1971,1972,1973]
# score_1_to_12 = list(range(1975,2023))

In [52]:
years = df_scores['Year'].unique()
normalized_scores = []

for year in years:
    year_scores = df_scores[df_scores['Year'] == year]
    
    scaled_scores = minmax_scale(df_scores[df_scores['Year'] == year]['Score'], feature_range=(1,12))
    scaled_scores = pd.DataFrame(scaled_scores, columns=['N_score'])
    normalized_scores.append(scaled_scores)

normalized_scores = pd.concat(normalized_scores)
df_scores['N_score'] = np.array(normalized_scores['N_score'])

In [53]:
# The data in the Eurovision website states Televoters points only from 2016, even due Televoters points were given starting 1997 in some of the countries.
#df_scores.pivot_table(index=['Year'], columns=['Score_type'], values=['Score'])

In [54]:
# Update the Score_type data according to the information on Wikipedia.
Voters = []

Voters.append({'Year': 1997, 'Format': 'Final', 'Score_type': 'Televoters', 'Countries': ['Austria', 'Switzerland', 'Germany', 'Sweden', 'United Kingdom']})
Voters.append({'Year': 1998, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Croatia', 'Greece', 'France', 'Spain', 'Slovakia', 'Poland', 'Israel', 'Germany', 'Malta', 'Slovenia', 'Ireland', 'Portugal',
                                 'United Kingdom', 'Cyprus', 'Netherlands', 'Sweden', 'Belgium', 'Finland', 'Norway', 'Estonia', 'North Macedonia']})
Voters.append({'Year': 1999, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Belgium', 'Spain', 'Croatia', 'United Kingdom', 'Slovenia', 'Norway', 'Denmark', 'France', 'Netherlands', 'Poland', 'Iceland',
                                 'Cyprus', 'Sweden', 'Portugal', 'Austria', 'Israel', 'Malta', 'Germany', 'Estonia']})
Voters.append({'Year': 2000, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Israel', 'United Kingdom', 'Estonia', 'France', 'Malta', 'Norway', 'Belgium', 'Cyprus', 'Iceland', 'Spain', 'Denmark', 'Germany',
                                 'Switzerland', 'Croatia', 'Sweden', 'Finland', 'Latvia', 'Ireland', 'Austria']})
Voters.append({'Year': 2001, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Netherlands', 'Iceland', 'Norway', 'Israel', 'Sweden', 'Lithuania', 'Latvia', 'Portugal', 'Ireland', 'Spain', 'France',
                                 'United Kingdom', 'Slovenia', 'Poland', 'Germany', 'Estonia', 'Denmark']})
Voters.append({'Year': 2001, 'Format': 'Final', 'Score_type': 'Combined', 'Countries': ['Croatia', 'Malta', 'Greece']})
Voters.append({'Year': 2002, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['United Kingdom', 'Austria', 'Estonia', 'Israel', 'Switzerland', 'Sweden', 'Denmark', 'Belgium', 'France', 'Germany', 'Latvia']})
Voters.append({'Year': 2002, 'Format': 'Final', 'Score_type': 'Combined',
                   'Countries': ['Cyprus', 'Greece', 'Spain', 'Croatia', 'Finland', 'Malta', 'Slovenia', 'Lithuania']})
Voters.append({'Year': 2003, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Iceland', 'Austria', 'Türkiye', 'Malta', 'Portugal', 'Croatia', 'Cyprus', 'Germany', 'Spain', 'Israel', 'Netherlands', 'Ukraine',
                                 'Greece', 'Norway', 'France', 'Poland', 'Latvia', 'Belgium', 'Estonia', 'Romania', 'Sweden', 'Slovenia']})
Voters.append({'Year': 2004, 'Format': 'Semi-Final', 'Score_type': 'Televoters',
                   'Countries': ['Spain', 'Austria', 'Norway', 'France', 'Serbia & Montenegro', 'Malta', 'Netherlands', 'Germany', 'Albania', 'Ukraine', 'Croatia',
                                 'Bosnia & Herzegovina', 'Belgium', 'Russia', 'North Macedonia', 'Greece', 'Iceland', 'Ireland', 'Poland', 'United Kingdom', 'Cyprus',
                                 'Türkiye', 'Romania', 'Sweden', 'Finland', 'Belarus', 'Latvia', 'Israel', 'Andorra', 'Portugal', 'Lithuania', 'Slovenia', 'Estonia', 'Denmark']})
Voters.append({'Year': 2004, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Spain', 'Austria', 'Norway', 'France', 'Serbia & Montenegro', 'Malta', 'Netherlands', 'Germany', 'Albania', 'Ukraine', 'Croatia',
                                 'Bosnia & Herzegovina', 'Belgium', 'Russia', 'North Macedonia', 'Greece', 'Iceland', 'Ireland', 'Poland', 'United Kingdom', 'Cyprus',
                                 'Türkiye', 'Romania', 'Sweden', 'Finland', 'Belarus', 'Latvia', 'Israel', 'Andorra', 'Portugal', 'Monaco', 'Lithuania', 'Slovenia',
                                 'Estonia', 'Denmark']})
Voters.append({'Year': 2005, 'Format': 'Semi-Final', 'Score_type': 'Televoters',
                   'Countries': ['Hungary', 'United Kingdom', 'Malta', 'Romania', 'Norway', 'Türkiye', 'Moldova', 'Cyprus', 'Spain', 'Israel', 'Serbia & Montenegro', 'Denmark',
                                 'Sweden', 'North Macedonia', 'Ukraine', 'Germany', 'Croatia', 'Greece', 'Russia', 'Bosnia & Herzegovina', 'Switzerland', 'Latvia', 'France',
                                 'Austria', 'Lithuania', 'Portugal', 'Belarus', 'Netherlands', 'Iceland', 'Belgium', 'Estonia', 'Finland', 'Bulgaria', 'Ireland', 'Slovenia', 'Poland']})
Voters.append({'Year': 2005, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Hungary', 'United Kingdom', 'Malta', 'Romania', 'Norway', 'Türkiye', 'Albania', 'Cyprus', 'Spain', 'Israel', 'Serbia & Montenegro', 'Denmark',
                                 'Sweden', 'North Macedonia', 'Ukraine', 'Germany', 'Croatia', 'Greece', 'Russia', 'Bosnia & Herzegovina', 'Switzerland', 'Latvia', 'France',
                                 'Austria', 'Lithuania', 'Portugal', 'Belarus', 'Netherlands', 'Iceland', 'Belgium', 'Estonia', 'Finland', 'Bulgaria', 'Ireland', 'Slovenia', 'Poland']})
Voters.append({'Year': 2006, 'Format': 'Semi-Final', 'Score_type': 'Televoters',
                   'Countries': ['Switzerland', 'Moldova', 'Israel', 'Latvia', 'Norway', 'Spain', 'Malta', 'Germany', 'Denmark', 'Russia', 'North Macedonia', 'Romania',
                                 'Bosnia & Herzegovina', 'Lithuania', 'United Kingdom', 'Greece', 'Finland', 'Ukraine', 'France', 'Croatia', 'Ireland', 'Sweden', 'Türkiye', 'Armenia',
                                 'Bulgaria', 'Slovenia', 'Andorra', 'Belarus','Belgium', 'Cyprus', 'Poland', 'Netherlands', 'Portugal', 'Estonia', 'Iceland']})
Voters.append({'Year': 2006, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Switzerland', 'Moldova', 'Israel', 'Latvia', 'Norway', 'Spain', 'Malta', 'Germany', 'Denmark', 'Russia', 'North Macedonia', 'Romania',
                                 'Bosnia & Herzegovina', 'Lithuania', 'United Kingdom', 'Greece', 'Finland', 'Ukraine', 'France', 'Croatia', 'Ireland', 'Sweden', 'Türkiye', 'Armenia',
                                 'Bulgaria', 'Slovenia', 'Andorra', 'Belarus','Belgium', 'Cyprus', 'Poland', 'Netherlands', 'Portugal', 'Estonia', 'Iceland']})
Voters.append({'Year': 2007, 'Format': 'Semi-Final', 'Score_type': 'Televoters',
                   'Countries': ['Bosnia & Herzegovina', 'Spain', 'Belarus', 'Ireland', 'Finland', 'North Macedonia', 'Slovenia', 'Hungary', 'Lithuania', 'Greece', 'Georgia', 'Sweden',
                                 'France', 'Latvia', 'Russia', 'Germany', 'Serbia', 'Ukraine', 'United Kingdom', 'Romania', 'Bulgaria', 'Türkiye', 'Armenia', 'Moldova', 'Israel',
                                 'Cyprus', 'Iceland', 'Montenegro', 'Switzerland', 'Netherlands', 'Denmark', 'Croatia', 'Poland', 'Czech Republic', 'Portugal', 'Norway', 'Malta',
                                 'Estonia', 'Belgium', 'Austria']})
Voters.append({'Year': 2007, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Bosnia & Herzegovina', 'Spain', 'Belarus', 'Ireland', 'Finland', 'North Macedonia', 'Slovenia', 'Hungary', 'Lithuania', 'Greece', 'Georgia', 'Sweden',
                                 'France', 'Latvia', 'Russia', 'Germany', 'Serbia', 'Ukraine', 'United Kingdom', 'Romania', 'Bulgaria', 'Türkiye', 'Armenia', 'Moldova', 'Israel',
                                 'Cyprus', 'Iceland', 'Montenegro', 'Switzerland', 'Netherlands', 'Denmark', 'Croatia', 'Poland', 'Czech Republic', 'Portugal', 'Norway', 'Malta',
                                 'Estonia', 'Belgium', 'Austria']})
Voters.append({'Year': 2008, 'Format': 'Semi-Final', 'Score_type': 'Televoters',
                   'Countries': ['Montenegro', 'Israel', 'Estonia', 'Moldova', 'Belgium', 'Azerbaijan', 'Slovenia', 'Norway', 'Poland', 'Ireland', 'Andorra', 'Bosnia & Herzegovina',
                                 'Armenia', 'Netherlands', 'Finland', 'Romania', 'Russia', 'Greece', 'Iceland', 'Sweden', 'Türkiye', 'Ukraine', 'Lithuania', 'Albania', 'Switzerland',
                                 'Czech Republic', 'Belarus', 'Latvia', 'Croatia', 'Bulgaria', 'Denmark', 'Georgia', 'Hungary', 'Malta', 'Cyprus', 'North Macedonia', 'Portugal',
                                 'United Kingdom', 'Germany', 'France', 'Spain', 'Serbia']})
Voters.append({'Year': 2008, 'Format': 'Final', 'Score_type': 'Televoters',
                   'Countries': ['Montenegro', 'Israel', 'Estonia', 'Moldova', 'Belgium', 'Azerbaijan', 'Slovenia', 'Norway', 'Poland', 'Ireland', 'Andorra', 'Bosnia & Herzegovina',
                                 'Armenia', 'Netherlands', 'Finland', 'Romania', 'Russia', 'Greece', 'Iceland', 'Sweden', 'Türkiye', 'Ukraine', 'Lithuania', 'Albania', 'Switzerland',
                                 'Czech Republic', 'Belarus', 'Latvia', 'Croatia', 'Bulgaria', 'Denmark', 'Georgia', 'Hungary', 'Malta', 'Cyprus', 'North Macedonia', 'Portugal',
                                 'United Kingdom', 'Germany', 'France', 'Spain', 'Serbia']})
Voters.append({'Year': 2009, 'Format': 'Semi-Final', 'Score_type': 'Televoters',
                   'Countries': ['Montenegro', 'Belgium', 'Belarus', 'Sweden', 'Armenia', 'Andorra', 'Switzerland', 'Türkiye', 'Israel', 'Bulgaria', 'Iceland', 'North Macedonia',
                                 'Romania', 'Finland', 'Portugal', 'Malta', 'Bosnia & Herzegovina', 'Croatia', 'Ireland', 'Latvia', 'Serbia', 'Poland', 'Norway', 'Cyprus', 'Slovakia',
                                 'Denmark', 'Slovenia', 'Hungary', 'Azerbaijan', 'Greece', 'Lithuania', 'Moldova', 'Ukraine', 'Estonia', 'Netherlands', 'France', 'Russia', 'Germany',
                                 'United Kingdom']})
Voters.append({'Year': 2009, 'Format': 'Final', 'Score_type': 'Combined',
                   'Countries': ['Montenegro', 'Belgium', 'Belarus', 'Sweden', 'Armenia', 'Andorra', 'Switzerland', 'Türkiye', 'Israel', 'Bulgaria', 'Iceland', 'North Macedonia',
                                 'Romania', 'Finland', 'Portugal', 'Malta', 'Bosnia & Herzegovina', 'Croatia', 'Ireland', 'Latvia', 'Serbia', 'Poland', 'Cyprus', 'Slovakia', 'Denmark',
                                 'Slovenia', 'Hungary', 'Azerbaijan', 'Greece', 'Lithuania', 'Moldova', 'Albania', 'Ukraine', 'Estonia', 'Netherlands', 'France', 'Russia', 'Germany',
                                 'United Kingdom', 'Spain']})
Voters.append({'Year': 2010, 'Format': 'Semi-Final', 'Score_type': 'Combined', 'Countries': df_scores[df_scores['Year']==2010]['Country'].unique()})
Voters.append({'Year': 2010, 'Format': 'Final', 'Score_type': 'Combined', 'Countries': df_scores[df_scores['Year']==2010]['Country'].unique()})
Voters.append({'Year': 2011, 'Format': 'Semi-Final', 'Score_type': 'Combined', 'Countries': df_scores[df_scores['Year']==2011]['Country'].unique()})
Voters.append({'Year': 2011, 'Format': 'Final', 'Score_type': 'Combined', 'Countries': df_scores[df_scores['Year']==2011]['Country'].unique()})
Voters.append({'Year': 2012, 'Format': 'Semi-Final', 'Score_type': 'Combined', 'Countries': df_scores[df_scores['Year']==2012]['Country'].unique()})
Voters.append({'Year': 2012, 'Format': 'Final', 'Score_type': 'Combined', 'Countries': df_scores[df_scores['Year']==2012]['Country'].unique()})
Voters.append({'Year': 2013, 'Format': 'Semi-Final', 'Score_type': 'Combined', 'Countries': df_scores[df_scores['Year']==2013]['Country'].unique()})
Voters.append({'Year': 2013, 'Format': 'Final', 'Score_type': 'Combined', 'Countries': df_scores[df_scores['Year']==2013]['Country'].unique()})
Voters.append({'Year': 2014, 'Format': 'Semi-Final', 'Score_type': 'Combined',
                   'Countries': ['Armenia', 'Latvia', 'Estonia', 'Sweden', 'Iceland', 'Russia', 'Azerbaijan', 'Ukraine', 'Belgium', 'Portugal', 'Netherlands', 'Hungary',
                                 'Malta', 'Israel', 'Norway', 'Poland', 'Austria', 'Lithuania', 'Finland', 'Ireland', 'Belarus', 'Switzerland', 'Greece', 'Slovenia',
                                 'Romania', 'Germany', 'France', 'Italy', 'Spain', 'Denmark', 'United Kingdom']})
Voters.append({'Year': 2014, 'Format': 'Final', 'Score_type': 'Combined',
                   'Countries': ['Armenia', 'Latvia', 'Estonia', 'Sweden', 'Iceland', 'Russia', 'Azerbaijan', 'Ukraine', 'Belgium', 'Moldova', 'Portugal', 'Netherlands',
                                 'Montenegro', 'Hungary', 'Malta', 'Israel', 'Norway', 'Poland', 'Austria', 'Lithuania', 'Finland', 'Ireland', 'Belarus', 'North Macedonia',
                                 'Switzerland', 'Greece', 'Slovenia', 'Romania', 'Germany', 'France', 'Italy', 'Spain', 'Denmark', 'United Kingdom']})
Voters.append({'Year': 2014, 'Format': 'Final', 'Score_type': 'Televoters', 'Countries': ['Georgia']})
Voters.append({'Year': 2015, 'Format': 'Semi-Final', 'Score_type': 'Combined',
                   'Countries': ['Moldova', 'Armenia', 'Belgium', 'Netherlands', 'Finland', 'Greece', 'Estonia', 'North Macedonia', 'Serbia', 'Hungary', 'Belarus', 'Russia',
                                 'Denmark', 'Albania', 'Romania', 'Lithuania', 'Ireland', 'Malta', 'Norway', 'Portugal', 'Czech Republic', 'Israel', 'Latvia', 'Iceland', 'Sweden',
                                 'Switzerland', 'Cyprus', 'Slovenia', 'Poland', 'France', 'United Kingdom', 'Australia', 'Spain', 'Italy']})
Voters.append({'Year': 2015, 'Format': 'Final', 'Score_type': 'Combined',
                   'Countries': ['Moldova', 'Armenia', 'Belgium', 'Netherlands', 'Finland', 'Greece', 'Estonia', 'Serbia', 'Hungary', 'Belarus', 'Russia', 'Denmark', 'Albania',
                                 'Romania', 'Georgia', 'Lithuania', 'Ireland', 'Malta', 'Norway', 'Portugal', 'Czech Republic', 'Israel', 'Latvia', 'Azerbaijan', 'Iceland', 'Sweden',
                                 'Switzerland', 'Cyprus', 'Slovenia', 'Poland', 'France', 'United Kingdom', 'Australia', 'Spain', 'Italy']})
Voters.append({'Year': 2015, 'Format': 'Final', 'Score_type': 'Televoters', 'Countries': ['North Macedonia', 'Montenegro']})

Voters = pd.DataFrame(Voters).set_index('Year')

for i,row in Voters.iterrows():
    df_scores.loc[(df_scores['Year']==i) & (df_scores['Format']==row['Format']) & (df_scores['From'].isin(row['Countries'])),'Score_type'] = row['Score_type']  
    
df_scores.loc[df_scores['Year']>=2016,'Score_type'] = 'Combined'

In [55]:
df_scores.to_csv('Eurovision score normalized.csv', index=False)